In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os


def is_colab():
    try:
        import google.colab

        return True
    except ImportError:
        return False


COLAB_ROOT_PATH = "/content/drive/MyDrive/final_project"

if is_colab():
    from google.colab import drive
    import sys

    drive.mount("/content/drive")
    if os.path.exists(COLAB_ROOT_PATH):
        os.chdir(COLAB_ROOT_PATH)
        sys.path.append(COLAB_ROOT_PATH)
    else:
        print(f"{COLAB_ROOT_PATH} is not exist")

    print("Running on Google Colab")
else:
    print("Not running on Google Colab")


!pip install -r requirements.txt

Mounted at /content/drive
Running on Google Colab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 3.4 MB/s eta 0:00:00


# Final Project Problem 2

![Image](./image.png)

In this jupyter notebook, we train a `neural network-based model` on the time series dataset from Problem 1.  
The conditions are as follows:

- Must use the same feature engineering method as the final model in problem 1 (since our goal is to compare ensemble methods vs neural networks on the same dataset)
- Use pytorch freely to build the model, as used in assignments #3 and #4
- Specifically:

  1. Use `problem_2.modeling.PandasDataset` to wrap the pandas dataset into a torch Dataset
  2. Build your model freely in `problem_2.modeling.load_model`
     - Input shape must match the features of final_train_dataset
     - Feel free to use anything including MLP or transformers
  3. Define optimizer and scheduler in `problem_2.modeling.load_optimizer` and `problem_2.modeling.load_scheduler`
     - Can use any optimizer defined in torch (ref: [link](https://docs.pytorch.org/docs/stable/optim.html#algorithms))
     - Can use any scheduler provided by torch (ref: [link](https://docs.pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate))
  4. Write training code in `problem_2.modeling.train`
     - Can write internal code freely but **must not modify the args**:

  ```python
  def train(
    model: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler.LRScheduler,
    train_loader: torch.utils.data.DataLoader,
    val_loader: torch.utils.data.DataLoader,
    device: Optional[torch.device] = None,
    ) -> torch.nn.Module:
    ....
  ```

  5. Model evaluation will be done using the `validate` function below


In [3]:
"""
The code below evaluates and saves the trained model.
Do not modify under any circumstances.
"""
import json
from typing import Callable, Optional, Tuple
import numpy as np
from torchmetrics.regression import MeanSquaredError
import torch
from pathlib import Path

from problem_2.modeling import get_device


def validate(
    model: torch.nn.Module,
    val_loader: torch.utils.data.DataLoader,
    invese_y: Callable[[np.ndarray], np.ndarray],
    device: Optional[torch.device] = None,
):
    device = device or get_device()
    mse = MeanSquaredError()
    model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            pred_y = model(x)

            invese_y_pred = torch.tensor(invese_y(pred_y.cpu().numpy().ravel()))
            invese_y_true = torch.tensor(invese_y(y.cpu().numpy().ravel()))
            mse.update(invese_y_pred, invese_y_true)

    return mse.compute()


def save_model(
    model: torch.nn.Module,
    optim: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler.LRScheduler,
    test_loader: torch.utils.data.DataLoader,
    train_loader: torch.utils.data.DataLoader,
    invese_y: Callable[[np.ndarray], np.ndarray],
    dst_path: str = "problem_2_cache",
    model_name: str = "trained_model",
) -> str:
    train_performance = validate(model, train_loader, invese_y)
    train_batch_size = train_loader.batch_size
    print(f"train performance: {train_performance}")
    test_performance = validate(model, test_loader, invese_y)
    test_batch_size = test_loader.batch_size
    print(f"test performance: {test_performance}")

    dst_path = (
        Path(dst_path)
        / f"{model_name}_tr_{train_performance:.4f}_te_{test_performance:.4f}_bs_{train_batch_size}_{test_batch_size}"
    )
    if dst_path.exists():
        cnt_exist = len(list(dst_path.parent.glob(f"{model_name}*")))
        version = cnt_exist + 1
        dst_path = dst_path.parent / f"{model_name}_{version}"
    dst_path.mkdir(parents=True, exist_ok=True)
    print(f"saving model to {dst_path}")

    torch.save(model.state_dict(), dst_path / "model.pth")
    torch.save(optim.state_dict(), dst_path / "optim.pth")
    torch.save(scheduler.state_dict(), dst_path / "scheduler.pth")
    with open(dst_path / "performance.json", "w") as f:
        json.dump(
            {
                "tr_performance": train_performance.item(),
                "te_performance": test_performance.item(),
            },
            f,
        )
    return dst_path


def load_trained_model(
    model_path: str,
    load_model: Callable[..., torch.nn.Module],
    load_optimizer: Callable[..., torch.optim.Optimizer],
    load_scheduler: Callable[..., torch.optim.lr_scheduler.LRScheduler],
) -> Tuple[
    torch.nn.Module,
    torch.optim.Optimizer,
    torch.optim.lr_scheduler.LRScheduler,
    float,
    float,
]:
    path_pth = Path(model_path)
    model_path = path_pth / "model.pth"
    optim_path = path_pth / "optim.pth"
    scheduler_path = path_pth / "scheduler.pth"
    performance_path = path_pth / "performance.json"
    with open(performance_path, "r") as f:
        performance = json.load(f)
    tr_performance = performance["tr_performance"]
    te_performance = performance["te_performance"]
    model = load_model()
    model.load_state_dict(torch.load(model_path, weights_only=True))
    optim = load_optimizer(model)
    optim.load_state_dict(torch.load(optim_path, weights_only=True))
    scheduler = load_scheduler(optim)
    scheduler.load_state_dict(torch.load(scheduler_path, weights_only=True))
    return model, optim, scheduler, tr_performance, te_performance


In [4]:
from feature_engineering import final_feature_engineering
import pandas as pd

training_dataset = pd.read_csv("ml_2025_final_project_training_dataset.csv")
test_dataset = pd.read_csv("ml_2025_final_project_test_dataset.csv")


final_train_dataset, final_test_dataset, final_inverse_y = final_feature_engineering(
    training_dataset, test_dataset
)

In [5]:
from problem_2.modeling import PandasDataset
from torch.utils.data import DataLoader

train_batch_size = 256  # Feel free to modify
test_batch_size = 128  # Feel free to modify

training_dataset = PandasDataset(final_train_dataset, "y")
test_dataset = PandasDataset(final_test_dataset, "y")
training_loader = DataLoader(
    training_dataset, batch_size=train_batch_size, shuffle=True
)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)
sample_batch = next(iter(training_loader))
x, y = sample_batch
print(f"x.shape: {x.shape}")
print(f"y.shape: {y.shape}")


x.shape: torch.Size([256, 16])
y.shape: torch.Size([256])


## 1. Model Setup

---

- Please freely write your model in problem_2/modeling.py's `load_model`.
- Refer to sample_load_model and sampe_type_2_load_model in the same file.


In [44]:
from problem_2.modeling import sample_load_model, load_model

# example of loading model
# model = sample_load_model()
# please load your model here
model = load_model()
print(model)

TransformerModel(
  (encoder): Linear(in_features=16, out_features=128, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=128, out_features=1, bias=True)
)


## 2. Optimizer and Scheduler

---

- Please freely write your model in problem_2/modeling.py's `load_optimizer` and `load_optimizer`
- Refer to `sample_load_optimizer` and `sample_load_scheduler` in the same file.


In [45]:
from problem_2.modeling import (
    sample_load_optimizer,
    sample_load_scheduler,
    load_optimizer,
    load_scheduler,
)

# example of loading optimizer and scheduler
# optim = sample_load_optimizer(model=model)
# please load your optimizer here
optim = load_optimizer(model=model)
print(optim)

# lr_scheduler = sample_load_scheduler(optimizer=optim)
# please load your scheduler here
lr_scheduler = load_scheduler(optimizer=optim)
print(lr_scheduler)

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.01
)


## 3. Training

---

- Please freely write your model in problem_2/modeling.py's `train`
- Refer to `sample_train` in the same file.


In [46]:
from problem_2.modeling import sample_train, train

# example of training
# trained_model = sample_train(
#     model=model,
#     optimizer=optim,
#     scheduler=lr_scheduler,
#     train_loader=training_loader,
#     val_loader=test_loader,
# )
# please train your model here
trained_model = train(
    model=model,
    optimizer=optim,
    scheduler=lr_scheduler,
    train_loader=training_loader,
    val_loader=test_loader,
)

--- Training Started ---
Epoch 1/30 | Train Loss: 836.2438 | Val Loss: 103.0011
  -> Found new best model with Val Loss: 103.0011
Epoch 2/30 | Train Loss: 660.1600 | Val Loss: 52.8996
  -> Found new best model with Val Loss: 52.8996
Epoch 3/30 | Train Loss: 517.5525 | Val Loss: 18.9980
  -> Found new best model with Val Loss: 18.9980
Epoch 4/30 | Train Loss: 384.6187 | Val Loss: 8.7572
  -> Found new best model with Val Loss: 8.7572
Epoch 5/30 | Train Loss: 269.6905 | Val Loss: 28.5186
  -> Val Loss did not improve. Patience: 1/7
Epoch 6/30 | Train Loss: 180.8297 | Val Loss: 83.4913
  -> Val Loss did not improve. Patience: 2/7
Epoch 7/30 | Train Loss: 123.7628 | Val Loss: 177.4922
  -> Val Loss did not improve. Patience: 3/7
Epoch 8/30 | Train Loss: 87.7178 | Val Loss: 38.7513
  -> Val Loss did not improve. Patience: 4/7
Epoch 9/30 | Train Loss: 58.1623 | Val Loss: 12.4453
  -> Val Loss did not improve. Patience: 5/7
Epoch 10/30 | Train Loss: 40.3952 | Val Loss: 8.1996
  -> Found new b

## 4. Validation

---

- Measure the final performance of your model.
- Note. The internal test dataset will also be measured in the same way.

```python
validate(your_model, internal_test_loader, your_final_inverse_y)
```

- Therefore, no errors should occur.


In [47]:
validate(model, test_loader, final_inverse_y)

tensor(1.3023)

## 5. Save Trained Model

---

- Save your model and submit it

- This code is used to test your code as it is called and tested during grading, so it must work and no modifications to the code are allowed.


In [48]:
save_path = save_model(
    model=model,
    optim=optim,
    scheduler=lr_scheduler,
    train_loader=training_loader,
    test_loader=test_loader,
    invese_y=final_inverse_y,
)

train performance: 0.9884642362594604
test performance: 1.3023041486740112
saving model to problem_2_cache/trained_model_tr_0.9885_te_1.3023_bs_256_128


## 6. Load trained Model

---

Load your saved model


In [50]:
model, optim, scheduler, tr_performance, te_performance = load_trained_model(
    model_path=save_path,
    load_model=load_model,
    load_optimizer=load_optimizer,
    load_scheduler=load_scheduler,
)
print(model)
print(optim)
print(scheduler)
print(tr_performance)
print(te_performance)

TransformerModel(
  (encoder): Linear(in_features=16, out_features=128, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=128, out_features=1, bias=True)
)
AdamW (
Parameter Group 0
    amsgrad: False
   

### Problem Summary

- Final submission file structure
- problem_2 /
  - best_model
    - model.pth
    - optim.pth
    - performance.json
    - scheduler.pth
- modeling.py
